# Query some information from the triples

As a reminder, here are our compentency questions:


| **ID** | **Competency question**                                                  | **Reference** |
|--------|--------------------------------------------------------------------------|---------------|
| 1      | In what language is written a song?                                      | Core          |
| 2      | Which different languages the lyrics from our Knowledge Graph use?       | Core          |
| 3      | Who is the primary artist involved in the creative process of a song?    | Music Meta    |
| 4      | What are all the different artists involved in writing the lyrics?       | Music Meta    |
| 5      | Which are the different sections of a song?                              | Music Lyrics  |
| 6      | What are the lyrics associated to an specific section of a song?         | Music Lyrics  |
| 7      | What are the starting and ending time stamps of a specific lyric's line? | Music Lyrics  |
| 8      | What is the duration of a lyric's line?                                  | Music Lyrics  |
| 9      | What is the duration of a song's section?                                | Music Lyrics  |
| 10     | What are the singers in charge of singing a specific song's section?     | Music Lyrics  |
| 11     | What is the dataset where the annotations were taken from?               | Music Lyrics  |
| 12     | From which dataset was extracted each JSON data file?                    | Music Lyrics  |

Let's try to answer them by performing some SPARQL queries in our knowledge graph's triples.

In [1]:
import os
from rdflib import Graph

In [2]:
# Directory containing the RDF files
triples_folder = './triples/'

# Load all RDF files into a single graph
graph = Graph()
for file_name in os.listdir(triples_folder):
    if file_name.endswith('.ttl') or file_name.endswith('.rdf'):
        graph.parse(os.path.join(triples_folder, file_name), format='turtle')

In [5]:
# SPARQL Queries
queries = {
    "song_language": """
        SELECT ?title ?language WHERE {
            ?song core:hasPart ?abstractScore ;
                core:title ?title .
            ?abstractScore mm:hasText ?lyrics .
            ?lyrics core:hasLanguage ?language .
        } LIMIT 25
    """,
    "different_languages": """
        SELECT DISTINCT ?language WHERE {
            ?song core:hasLanguage ?language .
        } LIMIT 25
    """,
    "primary_artist": """
        SELECT ?title ?artist WHERE {
            ?song core:title ?title ;
                mm:isCreatedBy ?creativeProcess .
            ?creativeProcess mm:involvesCreativeAction ?creativeActionURI .
            ?creativeActionURI mm:involvesAgent ?artistURI ;
                core:executesTask ?creativeTask .
            ?artistURI core:name ?artist .
            ?creativeTask rdf:type mm:Instrumentation
        } LIMIT 25
    """,
    "writer_artists": """
        SELECT ?title ?writerName WHERE {
            ?songURI core:title ?title ;
                    mm:isCreatedBy ?creativeProcess .
            ?creativeProcess mm:involvesCreativeAction ?creativeAction .
            ?creativeAction core:executesTask ?creativeTask ;
                            mm:involvesAgent ?writer .
            ?creativeTask a mm:LyricsWriting .
            ?writer core:name ?writerName .
        } LIMIT 25
    """,
    "song_sections": """
        SELECT DISTINCT ?section WHERE {
            ?lyrics mm:hasTextPart ?paragraph .
            ?paragraph a ?section .
        } 
    """,
    "lyrics_associated_to_section": """
        SELECT ?paragraph ?lyricsText WHERE {
            ?lyrics mm:hasTextPart ?paragraph .
            ?paragraph mm:hasTextPart ?textLine .
            ?textLine core:value ?lyricsText .
        } LIMIT 25
    """,
    "line_time_stamps": """
        SELECT ?lineText ?startValue ?endValue WHERE {
            ?lineURI a mm:TextFragment ;
                core:value ?lineText;
                ml:hasMusicTimeInterval ?intervalURI .
            ?intervalURI core:hasStartMusicTimeIndex ?startURI ;
                        core:hasEndMusicTimeIndex ?endURI .
            ?startURI core:hasMusicTimeIndexComponent ?start .
            ?endURI core:hasMusicTimeIndexComponent ?end .
            ?start core:value ?startValue .
            ?end core:value ?endValue .
        } LIMIT 25
    """,
    "line_duration": """
        SELECT ?lineText ?duration WHERE {
            ?lineURI a mm:TextFragment ;
                core:value ?lineText;
                ml:hasMusicTimeInterval ?intervalURI .
            ?intervalURI core:hasMusicTimeDuration ?durationURI .
            ?durationURI core:value ?duration .
        } LIMIT 25
    """,
    "section_duration": """
        SELECT DISTINCT ?paragraph ?duration WHERE {
            ?lyrics mm:hasTextPart ?paragraph .
            ?paragraph ml:hasMusicTimeInterval ?musicTimeInterval .
            ?musicTimeInterval core:hasMusicTimeDuration ?musicTimeDuration .
            ?musicTimeDuration core:value ?duration.
        } LIMIT 25
    """,
    "singers_for_section": """
        SELECT DISTINCT ?paragraph ?name WHERE {
            ?lyrics mm:hasTextPart ?paragraph .
            ?paragraph mm:hasTextPart ?textLine ;
                ml:sungBy ?musician .
            ?musician core:name ?name .
        } LIMIT 25
    """,
    "source_dataset": """
        SELECT ?songTitle ?datasetName WHERE {
            ?musicEntity core:title ?songTitle ;
                core:hasPart ?abstractScore .
            ?abstractScore prov:wasDerivedFrom ?sourceFile .
            ?sourceFile prov:wasDerivedFrom ?sourceDataset .
            ?sourceDataset core:name ?datasetName .
        } LIMIT 25 
    """,
    "source_file": """
        SELECT ?fileID ?datasetName WHERE {
            ?abstractScore prov:wasDerivedFrom ?JSONFile .
            ?JSONFile ml:hasIdentifier ?fileID ;
                        prov:wasDerivedFrom ?dataset .
            ?dataset core:name ?datasetName .
        } LIMIT 25
    """
}

In [6]:
# Execute queries and print results
for question, query in queries.items():
    print(f"\nResults for: {question.replace('_', ' ').title()}\n")
    results = graph.query(query)
    for row in results:
        print(row)


Results for: Song Language

(rdflib.term.Literal('I just fall in love again'), rdflib.term.Literal('en'))
(rdflib.term.Literal('One moment in time'), rdflib.term.Literal('en'))
(rdflib.term.Literal('If We Hold On Together'), rdflib.term.Literal('en'))
(rdflib.term.Literal("Can't Be With You Tonight"), rdflib.term.Literal('en'))
(rdflib.term.Literal('Darte un beso'), rdflib.term.Literal('es'))
(rdflib.term.Literal('El perdedor'), rdflib.term.Literal('es'))
(rdflib.term.Literal('Como la flor'), rdflib.term.Literal('es'))
(rdflib.term.Literal('El Poder Nuestro Es'), rdflib.term.Literal('es'))
(rdflib.term.Literal('Simplemente Amigos - Ana Gabriel'), rdflib.term.Literal('es'))
(rdflib.term.Literal('La oreja de van gogh- rosas'), rdflib.term.Literal('es'))
(rdflib.term.Literal('Fuego de noche, nieve de día'), rdflib.term.Literal('es'))
(rdflib.term.Literal('Entra en mi vida'), rdflib.term.Literal('es'))
(rdflib.term.Literal('Kilómetros'), rdflib.term.Literal('es'))
(rdflib.term.Literal('Pe